# Ensuring Consistency

**Activity Overview**: Ensure consistency by identifying and resolving conflicting values across datasets.

## Title: Customer Address Discrepancies

**Task**: Address customer address mismatches between CRM and marketing databases.

**Steps**:
1. Compare customer addresses in the CRM with those in the marketing database.
2. Identify records with conflicting address information.
3. Propose a method to consolidate records with verified addresses.

In [ ]:
# Write your code from here

In [1]:
import pandas as pd

# --- Task: Customer Address Discrepancies ---
print("--- Task: Customer Address Discrepancies ---")

# Load the datasets
try:
    crm_df = pd.read_csv('crm_customers.csv')
    marketing_df = pd.read_csv('erp_customers.csv')
except FileNotFoundError as e:
    print(f"Error loading file: {e}. Please ensure 'crm_customers.csv' and 'erp_customers.csv' are in the same directory.")
    # Create empty DataFrames to prevent further errors if files are missing
    crm_df = pd.DataFrame()
    marketing_df = pd.DataFrame()

if not crm_df.empty and not marketing_df.empty:
    # 1. Compare customer addresses in the CRM with those in the marketing database.
    # Merge the two DataFrames on 'customer_id' to bring addresses side-by-side
    # Use 'inner' merge to focus only on customers present in both systems.
    merged_customers_df = pd.merge(crm_df, marketing_df, on='customer_id', suffixes=('_crm', '_mkt'))

    # Normalize addresses for better comparison (e.g., lowercasing, removing extra spaces)
    # This step is crucial for real-world data where "Main St" vs "Main Street" might occur.
    # For simplicity, we'll just lowercase and strip spaces for this example.
    merged_customers_df['address_crm_norm'] = merged_customers_df['address_crm'].str.lower().str.strip()
    merged_customers_df['address_mkt_norm'] = merged_customers_df['address_mkt'].str.lower().str.strip()


    # 2. Identify records with conflicting address information.
    # A conflict exists if normalized addresses are different and neither is null.
    conflicting_addresses_df = merged_customers_df[
        (merged_customers_df['address_crm_norm'] != merged_customers_df['address_mkt_norm']) &
        (merged_customers_df['address_crm_norm'].notnull()) &
        (merged_customers_df['address_mkt_norm'].notnull())
    ].copy() # .copy() to avoid SettingWithCopyWarning

    if not conflicting_addresses_df.empty:
        print("\nCustomers with conflicting address information:")
        # Display relevant columns for conflicting records
        print(conflicting_addresses_df[['customer_id', 'name_crm', 'address_crm', 'address_mkt']])

        # 3. Propose a method to consolidate records with verified addresses.
        print("\n--- Proposed Methods for Consolidation with Verified Addresses ---")
        print("For each conflicting record, a common approach is to:")
        print("1. **Manual Review and Correction:** The most reliable method for critical data like addresses. Data stewards or support teams would manually check each conflicting address against a verified source (e.g., postal service database, customer contact) and update the correct system.")
        print("2. **Master Data Management (MDM) System:** Implement an MDM solution that acts as a central repository for golden customer records. It would define rules for merging and prioritizing data from different sources.")
        print("3. **Source Priority Rule:** If one source is inherently more reliable for addresses (e.g., CRM is the system of record for customer details), automatically prioritize its address. For example, always use CRM address if conflict:")
        # Example of applying a source priority rule (e.g., prioritize CRM address)
        # Note: This is an example of the *logic* for consolidation, not modifying the original DFs
        print("\n   Example of prioritizing CRM address:")
        # Create a new column for the 'resolved' address based on a priority rule
        def resolve_address(row):
            if row['address_crm_norm'] == row['address_mkt_norm']:
                return row['address_crm'] # They match, pick one
            else:
                # If they conflict, prioritize CRM address
                return row['address_crm']

        # Apply this logic to a subset of conflicting data for demonstration
        # In a real scenario, you'd apply this to the full merged_customers_df or to your master customer data
        # For demonstration, let's create a hypothetical resolved column for these conflicts
        conflicting_addresses_df['resolved_address'] = conflicting_addresses_df.apply(resolve_address, axis=1)
        print(conflicting_addresses_df[['customer_id', 'address_crm', 'address_mkt', 'resolved_address']])
        print("\n   This 'resolved_address' would then be used to update the less accurate system or a master record.")

        print("4. **Consensus-based Approach:** If multiple sources exist, choose the address that appears most frequently or has been recently updated.")
        print("5. **Data Cleansing/Validation Services:** Integrate with external address validation APIs (e.g., Google Maps API, USPS address validation) to verify and standardize addresses before consolidation.")

    else:
        print("No conflicting address information found for customers present in both datasets.")

else:
    print("Skipping customer address discrepancies check due to file loading error.")

--- Task: Customer Address Discrepancies ---


KeyError: 'address_crm'